In [1]:
!pip install -q streamlit
!pip install transformers
!pip install accelerator
!pip install langchain-huggingface
!pip install langchain_community
!pip install langchain huggingface_hub
!pip install pypdf
!pip install faiss-gpu
!pip install langchain_core
!pip install ctransformers
!pip install langchain
!pip install -qU langchain-text-splitters
!pip install numpy
!npm install -g localtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.5/366.5 kB 8.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
 

In [2]:
!npm install -g localtunnel

m#################.] \ reify:yargs-parser: timing reifyNode:node_modules/localt
changed 22 packages in 633ms

3 packages are looking for funding
  run `npm fund` for details


In [3]:
import streamlit as st
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import CTransformers
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret('HF_TOKEN')
HUGGINGFACEHUB_API_KEY = user_secrets.get_secret('HUGGINGFACEHUB_API_KEY')
LANGCHAIN_API_KEY = user_secrets.get_secret('LANGCHAIN_API_KEY')

In [5]:
def HuggingFace_Embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = 'mixedbread-ai/mxbai-embed-large-v1', 
                                      model_kwargs = {'device' : 'GPU T4 x 2'}, 
                                      encode_kwargs = {'normalize_embeddings' : True})
    return embeddings


llm = CTransformers(model="TheBloke/Llama-2-13B-Ensemble-v5-GGUF", 
                    model_file="llama-2-13b-ensemble-v5.Q5_K_M.gguf",
                    model_type="llama",
                    gpu_layers = 20)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-13b-ensemble-v5.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [6]:
%%writefile app.py
import streamlit as st
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import CTransformers
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain




from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret('HF_TOKEN')
HUGGINGFACEHUB_API_KEY = user_secrets.get_secret('HUGGINGFACEHUB_API_KEY')
LANGCHAIN_API_KEY = user_secrets.get_secret('LANGCHAIN_API_KEY')




def HuggingFace_Embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = 'mixedbread-ai/mxbai-embed-large-v1', 
                                      model_kwargs = {'device' : 'cuda'}, 
                                      encode_kwargs = {'normalize_embeddings' : True})
    return embeddings


llm = CTransformers(model="TheBloke/Llama-2-13B-Ensemble-v5-GGUF", 
                    model_file="llama-2-13b-ensemble-v5.Q5_K_M.gguf",
                    model_type="llama",
                    gpu_layers = 20)






def session_state():
    
    if 'vectors' not in st.session_state:
        
        st.session_state.embeddings = HuggingFace_Embeddings()
        st.session_state.loader = PyPDFDirectoryLoader('/kaggle/input/documents/Law documents')
        st.session_state.documents = st.session_state.loader.load()
        st.session_state.text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
        st.session_state.final_documents = st.session_state.text_splitter.split_documents(st.session_state.documents)
        st.session_state.vectors = FAISS.from_documents(st.session_state.final_documents, st.session_state.embeddings)
        st.session_state.vectors.save_local('faiss_index')
        st.session_state.new_db = FAISS.load_local('faiss_index', st.session_state.embeddings, allow_dangerous_deserialization = True)

# %% [code] {"jupyter":{"outputs_hidden":false}}
st.title('Law guidence with llama2')
prompt = ChatPromptTemplate.from_template(
                                          """
                                          Answer the questions based on the provided context only.
                                          understand the question and give the correct law advise and suggestions. 
                                          please don't share false information.
                                          <context>
                                          {context}
                                          <context>
                                          Questions : {input}
                                          """
                                         )
input_prompt = st.text_input("Enter the question realted to LAW")
if st.button("Document embedding"):
    session_state()
    st.write("Database is ready..")

import time

if input_prompt:
    document_chain = create_stuff_documents_chain(llm, prompt)
    retriever = st.session_state.vectors.as_retriever()
    retrieval_chain = create_retrieval_chain(retriever, document_chain)
    start=time.process_time()
    
    response = retrieval_chain.invoke({'input':input_prompt})

    print("Response time :",time.process_time()-start)
    st.write(response['answer'])

    # With a streamlit expander
    with st.expander("Document Similarity Search"):
        # Find the relevant chunks
        for i, doc in enumerate(response["context"]):
            st.write(doc.page_content)
            st.write("--------------------------------")

Writing app.py


In [7]:
!cat app.py

import streamlit as st
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import CTransformers
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain




from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret('HF_TOKEN')
HUGGINGFACEHUB_API_KEY = user_secrets.get_secret('HUGGINGFACEHUB_API_KEY')
LANGCHAIN_API_KEY = user_secrets.get_secret('LANGCHAIN_API_KEY')




def HuggingFace_Embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = 'mixedbread-ai/mxbai-embed-large-v1', 
                                      model_kwargs = {'device' : 'cuda'}, 
                             

In [8]:
! wget -q -O - ipv4.icanhazip.com

104.199.164.84


In [10]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://104.199.164.84:8501

your url is: https://chilly-windows-clean.loca.lt
Fetching 1 files: 100%|█████████████████████████| 1/1 [00:00<00:00, 5440.08it/s]
2024-07-16 14:15:14.832523: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-16 14:15:14.832585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-16 14:15:14.834083: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Fetching 1 files: 100%|█████████████████████████| 1/1 [00: